In [ ]:
import zkbp
import os,sys
from pathlib import Path
path = os.path.realpath('0-zkbp-intro.ipynb')
sys.path.append(str(Path(path).parents[1]))  # go up 2 levels [1] to '/pyledger/'
from pyledger.zkutils import Commit,r_blend, Token

In [ ]:
# generate g and h
gh = zkbp.gen_GH()

# generate public and secret key
pk_sk_obj = zkbp.gen_pb_sk(gh)
pk = pk_sk_obj.get_pk()
sk = pk_sk_obj.get_sk()
print(f'public key is {pk} and secret key is {sk}')

# generate a random point
r = zkbp.gen_r()
# print(f'r is {r.get}')

# This r does not have add,mult,neg functions, so we use class r_blend()
r = r_blend()
rd = -r 

In [ ]:
# lets make a commitment
# we can make a Pedersen commitment, we can do that with a commit object
value = 13
r = zkbp.gen_r()
cm = zkbp.commit(value, r, gh)
#print(cm.get)

# another option
r = r_blend()
cmd = Commit(gh, value, r)
print(cmd.eval.get)

# we can also create a token that is just the public key raised to r
tk = pk_sk_obj.to_token(r.val)
print(tk.get)

In [ ]:
# proof of knowledge: lets create a zk proof and then verify it

#g2val = zkbp.g_to_x(gh,value)
sval = 10
r = r_blend()
cm = Commit(gh,sval,r)
tk = pk_sk_obj.to_token(r.val)
#h2r = Commit(gh,0,r).eval
#tk = pk_sk_obj.to_token(r.val)
h2r = zkbp.sub(zkbp.from_str(cm.eval.get), zkbp.from_str(zkbp.g_to_x(gh, sval).get))
pr = zkbp.sigma_dlog_proof_explicit_sha256(tk,pk_sk_obj,h2r)
t = zkbp.sigma_dlog_proof_explicit_verify_sha256(pr,h2r,tk)

print(pr)
print(t)
# hash order: pk_t_rand_commitment, h2r, pk

In [ ]:
# lets create a range proof
v = 10 #(try -1, 1 )
r = r_blend()
gh = zkbp.gen_GH()
cm = Commit(gh, v, r)
n_bit = 4 #(try 1, 2)
pr = zkbp.range_proof_single(n_bit=n_bit, val=v, gh=gh, r=r.val)
res = zkbp.range_proof_single_verify(pr, n_bit, gh, cm.eval)
print(pr)
print(res)

In [ ]:
# finally lets try a discrete log equality proof 
# this proof lets you prove a balance .. 
# that sum of commits will open to balance and sum of tokens blinding factors (rs)
r = r_blend()
val = 10
tk = pk_sk_obj.to_token(r.val)
cm = Commit(gh,val,r).eval
h_sum_r = zkbp.sub(cm, zkbp.g_to_x(gh, val))
pr = zkbp.sigma_eq_dlog_proof_sha256(tk, pk_sk_obj, gh, h_sum_r)
cm = Commit(gh,val,r).eval
print(pr)
res = zkbp.sigma_eq_dlog_verify_sha256(pr, gh, h_sum_r, tk)
print(res)

In [ ]:
r = r_blend()
rd = -r 
newr = rd + r
v = 0
tk = pk_sk_obj.to_token(r.val)
print(f'tk is {tk.get}')
cm = Commit(gh,v,r).eval
print(f'cm is {cm.get}')
print(f'pk is {pk}')
pr = zkbp.consistency_proof(v,r.val,gh,cm,tk,pk)
print(pr)
result = zkbp.consistency_proof_verify(proof=pr, gh=gh, ped_cm=cm, token=tk,
                                               pubkey=pk_sk_obj.get_pk())
print(result)